In [1]:
import torch
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from getimagenetclasses import parseclasslabel, parsesynsetwords, get_classes
# Try other models https://pytorch.org/vision/stable/models.html
from torchvision.models import resnet18

In [2]:
class ImageNet2500(Dataset):
    def __init__(self, root_dir, xmllabeldir, synsetfile, images_dir, transform=None):

        """
    Args:

        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """

        self.root_dir = root_dir
        self.xmllabeldir = root_dir + xmllabeldir
        self.images_dir = root_dir + images_dir
        self.transform = transform
        self.imgfilenames = []
        self.labels = []
        self.ending = ".JPEG"

        indicestosynsets, self.synsetstoindices, synsetstoclassdescr = parsesynsetwords(root_dir + synsetfile)

        for file in os.listdir(self.images_dir):
            if file.endswith(".JPEG"):
                #name = os.path.join(images_dir, file)
                self.imgfilenames.append(file)


    def filenametoxml(self, fn):
        f = os.path.basename(fn)

        if not f.endswith(self.ending):
            print('not f.endswith(self.ending)')
            exit()

        f = f[:-len(self.ending)] + '.xml'
        f = os.path.join(self.xmllabeldir, f)

        return f
    
    def resize_and_crop(self, img, s=224, crop=True):
            width, height = img.size
            
            # Rescale smallest side to s
            if width > height:
                img = img.resize((int(s*(width/height)), int(s)))
            else:
                img = img.resize((int(s), int(s*(height/width))))

            if crop:
            # Center-cropping to (s x s)
                left = (img.size[0] - s) // 2
                top = (img.size[1] - s) // 2

                coords = (left, top, left+s, top+s)
                img = img.crop(coords)
        
            return img

    def __len__(self):
        # todo: return the number of samples in the dataset
        return len(self.imgfilenames)

    def __getitem__(self, idx):
        #  todo: load the image of index idx, transform it using the appropriate transforms and return it along with its
        #  label
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.images_dir, self.imgfilenames[idx])
        image = Image.open(img_name).convert("RGB")
        
        image = self.resize_and_crop(img=image, s=224, crop=True)

        if self.transform:
            image = self.transform(image)

        label, _ = parseclasslabel(self.filenametoxml(self.imgfilenames[idx]), self.synsetstoindices)
        #self.labels.append(label)

        return image, label

In [3]:
def run_model(model, dataloader):
    pred = torch.Tensor()
    lbls = torch.Tensor()

    for batch_idx, (images, labels) in enumerate(dataloader):
        outputs = model(images)

        pred = torch.cat((pred, outputs), dim=0)
        lbls = torch.cat((lbls, labels), dim=0)

    return pred, lbls


In [4]:
def plot_example(indx, model, dataset):
    sample = dataset[indx]
    plt.imshow(sample["image"].permute(1, 2, 0))
    plt.show()
    # im = transforms.ToPILImage()(sample["image"])
    # im.show()
    prediction = model(sample["image"].unsqueeze(0)).detach().numpy()[0]
    ind = prediction.argsort()[-5:][::-1]
    print("Top-5 predicted levels:\n")
    for key in ind:
        print(get_classes().get(key))

    print("\nTrue label ", get_classes()[sample["label"]])


In [5]:
def compare_performance(model, loader_wo_normalize, loader_w_normalize):
    # predictions and labels from dataset without normalization
    preds, labels = run_model(model, loader_wo_normalize)
    # predictions and labels from dataset with normalization (labels are the same as before)
    preds_norm, _ = run_model(model, loader_w_normalize)

    #  todo: calculate the accuracy when using normalized data and when using un-normalized data
    acc = None
    acc_norm = None

    print("Accuracy without normalize: ", acc)
    print("Accuracy with normalize: ", acc_norm)


In [6]:
main_path = "/mnt/e/ml_projects/IN3310/2025/exercise_data/week4_files/"
# These files/folders should be inside the main_path directory, i.e.
# ../solution /
# ├── ILSVRC2012_bbox_val_v3 /
# │   └── val /
# ├── imagenet2500 /
# │   └── imagespart /
# ├── getimagenetclasses.py
# └── synset_words.txt

xmllabeldir = "ILSVRC2012_bbox_val_v3/val/"
synsetfile = 'synset_words.txt'
images_dir = "imagenet2500/imagespart"

#  https://pytorch.org/vision/main/generated/torchvision.transforms.Compose.html
base_transform = transforms.Compose([
    transforms.ToTensor()
    #  use the appropriate transforms
])

normalize_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #  use the appropriate transforms in addition to normalization
    #  use these statistics for normalization:
    #  mean: [0.485, 0.456, 0.406], std_dev: [0.229, 0.224, 0.225]
])

#  todo: create a dataset and a dataloader without the normalization
dataset_wo_normalize = ImageNet2500(root_dir=main_path, 
                                    xmllabeldir=xmllabeldir, 
                                    synsetfile=synsetfile, 
                                    images_dir=images_dir, 
                                    transform=base_transform)
loader_wo_normalize = DataLoader(dataset_wo_normalize, batch_size=23, shuffle=True)

#  todo: create a dataset and a dataloader **with** normalization
dataset_w_normalize = ImageNet2500(root_dir=main_path, 
                                    xmllabeldir=xmllabeldir, 
                                    synsetfile=synsetfile, 
                                    images_dir=images_dir, 
                                    transform=normalize_transform)
loader_w_normalize = DataLoader(dataset_w_normalize, batch_size=23, shuffle=True)


In [7]:
# load a pretrained model of choice (see models in torchvision.models)
model = resnet18(pretrained=True)
# Set model to eval mode to use the learned-statistics instead of batch-statistics for batch_norm, and skip
# training-only operations like dropout. Try removing this line and see how the model performs!
model.eval()

/mnt/e/ml_projects/in3310/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/e/ml_projects/in3310/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
compare_performance(model, loader_wo_normalize, loader_w_normalize)
# change the index to check other examples
plot_example(6, model, dataset_wo_normalize)

: 